In [161]:

import nltk
import pyarrow as pa
import pyarrow.parquet as pq
import sklearn
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from sklearn.linear_model import LinearRegression

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [141]:
df = pd.read_parquet('yelp_final.parquet')

In [142]:
df


,business_id,city,state,latitude,longitude,stars,review_count,categories,Days,km2,km²,Porcentaje del PIB Nacional,Población (Millones),PIB per cápita ($),PIB per cápita Posición,total_sentimientos
239448,---kPU91CF4Lq2-WlRu9Lw,New Port Richey,Florida,28.217288,-82.733344,4.5,24.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,8.0
174001,--epgcb7xHGuJ-4PUeSLAw,Willow Grove,Florida,40.145054,-75.116293,3.0,34.0,Restaurant Sandwiches Fast Food,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,8.0
236126,-0EdehHjIQc0DtYU8QcAig,Land O Lakes,Florida,28.260154,-82.474816,3.0,35.0,Restaurant Chinese,None,145.0,138888.1,5.20,18.8,40.106,40,2.0
238473,-0FX23yAacC4bbLaGPvyxw,Newtown,Florida,40.256082,-74.916976,3.5,155.0,Restaurant Breakfast & Brunch American,"Tuesday, Wednesday, Thursday, Friday, Saturday...",145.0,138888.1,5.20,18.8,40.106,40,4.0
228308,-0M0b-XhtFagyLmsBtOe8w,Philadelphia,Florida,39.967439,-75.175452,3.5,18.0,Restaurant Breakfast & Brunch American,"Thursday, Friday, Saturday",145.0,138888.1,5.20,18.8,40.106,40,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276465,zyHP-oXgDkANEyQbJVKf8g,Nashville,California,36.129826,-86.778134,4.0,124.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",97.0,403932.0,13.34,37.3,51.914,12,28.0
183620,zyMkbavgHASQtqVwaock9A,Philadelphia,Florida,39.962976,-75.169315,4.0,21.0,Restaurant Coffee & Tea,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,6.0
232565,zymvTQ12tRy3bzmP4tPgOg,Saint Charles,Florida,38.777849,-90.483287,4.0,368.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,34.0
153442,zzIF9qp2UoHN48EeZH_IDg,Tarpon Springs,Florida,28.137368,-82.740846,3.0,19.0,Restaurant Chicken Wings - Fast Food,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,10.0


funcion  que nos dara el puntaje de acuerdo  sus pesos establecidos..... dandole mas importancia al sentimiento como peso principal en nuestro puntaje

In [143]:


def calcular_puntaje(fila):
    puntaje = 0
    
    pesos = {'total_sentimientos': -2, 'PIB per cápita ($)': 1, 'Población (Millones)': 1, 'review_count': 1, }
    for columna, peso in pesos.items():
        puntaje += peso * fila[columna]
    return puntaje

aplico la funcion y agrego a mi dataset

In [144]:

df['puntaje'] = df.apply(calcular_puntaje, axis=1)

In [145]:
df

,business_id,city,state,latitude,longitude,stars,review_count,categories,Days,km2,km²,Porcentaje del PIB Nacional,Población (Millones),PIB per cápita ($),PIB per cápita Posición,total_sentimientos,puntaje
239448,---kPU91CF4Lq2-WlRu9Lw,New Port Richey,Florida,28.217288,-82.733344,4.5,24.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,8.0,66.906
174001,--epgcb7xHGuJ-4PUeSLAw,Willow Grove,Florida,40.145054,-75.116293,3.0,34.0,Restaurant Sandwiches Fast Food,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,8.0,76.906
236126,-0EdehHjIQc0DtYU8QcAig,Land O Lakes,Florida,28.260154,-82.474816,3.0,35.0,Restaurant Chinese,None,145.0,138888.1,5.20,18.8,40.106,40,2.0,89.906
238473,-0FX23yAacC4bbLaGPvyxw,Newtown,Florida,40.256082,-74.916976,3.5,155.0,Restaurant Breakfast & Brunch American,"Tuesday, Wednesday, Thursday, Friday, Saturday...",145.0,138888.1,5.20,18.8,40.106,40,4.0,205.906
228308,-0M0b-XhtFagyLmsBtOe8w,Philadelphia,Florida,39.967439,-75.175452,3.5,18.0,Restaurant Breakfast & Brunch American,"Thursday, Friday, Saturday",145.0,138888.1,5.20,18.8,40.106,40,0.0,76.906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276465,zyHP-oXgDkANEyQbJVKf8g,Nashville,California,36.129826,-86.778134,4.0,124.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",97.0,403932.0,13.34,37.3,51.914,12,28.0,157.214
183620,zyMkbavgHASQtqVwaock9A,Philadelphia,Florida,39.962976,-75.169315,4.0,21.0,Restaurant Coffee & Tea,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,6.0,67.906
232565,zymvTQ12tRy3bzmP4tPgOg,Saint Charles,Florida,38.777849,-90.483287,4.0,368.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,34.0,358.906
153442,zzIF9qp2UoHN48EeZH_IDg,Tarpon Springs,Florida,28.137368,-82.740846,3.0,19.0,Restaurant Chicken Wings - Fast Food,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,10.0,57.906


In [149]:




# Define tus variables de entrada y salida
X = df[['puntaje']]
y = df[['latitude', 'longitude']]

# Divide tus datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crea y entrena tu modelo KNN
modelo = KNeighborsRegressor(n_neighbors=3)
modelo.fit(X_train, y_train)

# Haz predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)

# Calcula el error cuadrático medio de tus predicciones
mse = mean_squared_error(y_test, y_pred)
print(f'Error cuadrático medio: {mse}')

# Ahora puedes usar el modelo para hacer predicciones
nuevo_puntaje = 90  # sustituye esto por el puntaje de tu nuevo restaurante
prediccion = modelo.predict(np.array([[nuevo_puntaje]]))
print(f'Predicción de latitud y longitud para el nuevo restaurante: {prediccion}')



Error cuadrático medio: 9047461.913961457
Predicción de latitud y longitud para el nuevo restaurante: [[ 39.89104277 -78.81859183]]


C:\Users\coco\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


In [157]:
predicciones = modelo.predict(X)
df_predicciones = pd.DataFrame(predicciones, columns=['latitud_predicha', 'longitud_predicha'])

In [158]:
df_predicciones

,latitud_predicha,longitud_predicha
0,31.921491,-83.799377
1,39.476331,-83.969853
2,37.117718,-94.934401
3,39.862407,-78.858347
4,39.476331,-83.969853
...,...,...
12146,30.560748,-84.082570
12147,39.903519,-82.417569
12148,37.688135,-98.773467
12149,39.857192,-78.787348


In [159]:
# Restablece los índices de df y df_predicciones
df = df.reset_index(drop=True)
df_predicciones = df_predicciones.reset_index(drop=True)

# Ahora puedes agregar estas nuevas columnas a tu DataFrame original
df['latitud_predicha'] = df_predicciones['latitud_predicha']
df['longitud_predicha'] = df_predicciones['longitud_predicha']



In [160]:
df

,business_id,city,state,latitude,longitude,stars,review_count,categories,Days,km2,km²,Porcentaje del PIB Nacional,Población (Millones),PIB per cápita ($),PIB per cápita Posición,total_sentimientos,puntaje,latitud_predicha,longitud_predicha
0,---kPU91CF4Lq2-WlRu9Lw,New Port Richey,Florida,28.217288,-82.733344,4.5,24.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,8.0,66.906,31.921491,-83.799377
1,--epgcb7xHGuJ-4PUeSLAw,Willow Grove,Florida,40.145054,-75.116293,3.0,34.0,Restaurant Sandwiches Fast Food,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,8.0,76.906,39.476331,-83.969853
2,-0EdehHjIQc0DtYU8QcAig,Land O Lakes,Florida,28.260154,-82.474816,3.0,35.0,Restaurant Chinese,None,145.0,138888.1,5.20,18.8,40.106,40,2.0,89.906,37.117718,-94.934401
3,-0FX23yAacC4bbLaGPvyxw,Newtown,Florida,40.256082,-74.916976,3.5,155.0,Restaurant Breakfast & Brunch American,"Tuesday, Wednesday, Thursday, Friday, Saturday...",145.0,138888.1,5.20,18.8,40.106,40,4.0,205.906,39.862407,-78.858347
4,-0M0b-XhtFagyLmsBtOe8w,Philadelphia,Florida,39.967439,-75.175452,3.5,18.0,Restaurant Breakfast & Brunch American,"Thursday, Friday, Saturday",145.0,138888.1,5.20,18.8,40.106,40,0.0,76.906,39.476331,-83.969853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12146,zyHP-oXgDkANEyQbJVKf8g,Nashville,California,36.129826,-86.778134,4.0,124.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",97.0,403932.0,13.34,37.3,51.914,12,28.0,157.214,30.560748,-84.082570
12147,zyMkbavgHASQtqVwaock9A,Philadelphia,Florida,39.962976,-75.169315,4.0,21.0,Restaurant Coffee & Tea,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,6.0,67.906,39.903519,-82.417569
12148,zymvTQ12tRy3bzmP4tPgOg,Saint Charles,Florida,38.777849,-90.483287,4.0,368.0,Restaurant Breakfast & Brunch American,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,34.0,358.906,37.688135,-98.773467
12149,zzIF9qp2UoHN48EeZH_IDg,Tarpon Springs,Florida,28.137368,-82.740846,3.0,19.0,Restaurant Chicken Wings - Fast Food,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",145.0,138888.1,5.20,18.8,40.106,40,10.0,57.906,39.857192,-78.787348
